In [80]:
import random
import math
import heapq

In [82]:
def build_MST(plant_id, substation_positions):
    """
    Build a Minimum Spanning Tree (MST) over all substations using Prim's algorithm.

    The MST ensures that the full network is connected at minimum total Euclidean distance.
    This version of Prim starts at the designated plant node, although the final MST does
    not depend on the starting node. All edge weights are Euclidean distances between
    substation coordinates in the 2-D region.

    Args:
        plant_id (int): The index of the substation where Prim's algorithm begins.
        substation_positions (dict[int, tuple[float, float]]):
            A dictionary mapping each substation index to its (x, y) coordinates.

    Returns:
        set[tuple[int, int, float]]:
            A set of edges in the form (u, v, distance) describing the MST.
            Each edge is undirected and uses (min(u, v), max(u, v), distance)
            to avoid duplicates.
    """

    # Helper function to compute Euclidean distance between two substations
    def distance(u, v):
        x1, y1 = substation_positions[u]
        x2, y2 = substation_positions[v]
        return math.hypot(x1 - x2, y1 - y2)

    mst_edges = set()                   # Stores the MST edges
    visited = set()                     # Tracks nodes already added to the MST
    priority_queue = []                 # Min-heap of edges by distance
    num_substations = len(substation_positions)

    # Mark the plant as the starting node
    visited.add(plant_id)

    # Push all edges from the plant to its neighbors
    for neighbor in range(num_substations):
        if neighbor != plant_id:
            d = distance(plant_id, neighbor)
            heapq.heappush(priority_queue, (d, plant_id, neighbor))

    # Prim's main loop
    while len(visited) < num_substations:
        d, u, v = heapq.heappop(priority_queue)

        # Skip edges that lead to already-visited nodes
        if v in visited:
            continue

        # Add the new node and this edge to the MST
        visited.add(v)

        # Normalize ordering for undirected edges
        u2 = min(u, v)
        v2 = max(u, v)

        mst_edges.add((u2, v2, d))

        # Push all edges from v to unvisited nodes
        for w in range(num_substations):
            if w not in visited:
                dist_vw = distance(v, w)
                heapq.heappush(priority_queue, (dist_vw, v, w))

    return mst_edges

In [89]:
def make_graph(
    num_substations, 
    area_size = 100,
    min_cost_multiplier = 0.1, 
    max_cost_multiplier = 10, 
    k_nearest = 4,
    seed = None,
    file_name = "graph.txt", 
    plant_id = 0
):
    """
    Generate a realistic synthetic power transmission network.

    Substations are placed randomly in a 2-D square region of size `area_size` × `area_size`.
    Each substation attempts to connect to its k geographically nearest neighbors. An internal
    minimum spanning tree (MST) is added to ensure global connectivity. Edge construction costs
    are assigned as:
        cost = distance * random.uniform(min_cost, max_cost)

    The resulting graph is written to a file in edge-list format.

    Args:
        num_substations (int): Number of substations (nodes) in the network.
        area_size (float): Side length of the 2-D simulation region in miles.
        min_cost (float): Minimum cost-per-mile multiplier for line construction.
        max_cost (float): Maximum cost-per-mile multiplier for line construction.
        k_nearest (int): Number of nearest neighbors each substation attempts to connect to.
        seed (int, optional): Random seed for reproducibility.
        file_name (str): Output filename for the edge list.
        plant_id (int): Substation that will represent the generation plant.

    Returns:
        None. Writes the generated graph to `file_name`.
    """
    
    # Helper function to compute Euclidean distance between two substations
    def distance(u, v):
        x1, y1 = substation_positions[u]
        x2, y2 = substation_positions[v]
        return math.hypot(x1 - x2, y1 - y2)
    
    # Initialize RNG
    if seed is not None:
        random.seed(seed)
    
    # Distribute substations across 2-D square
    substation_positions = {}
    for i in range(num_substations):
        if i == plant_id:
        # Plant will be placed in a location near the center of the square
            cx = area_size / 2
            cy = area_size / 2
            x_coordinate = random.normalvariate(cx, area_size * 0.01)
            y_coordinate = random.normalvariate(cy, area_size * 0.01)
        
        else: 
        # Distribute all other substations randomly
            x_coordinate = random.uniform(0, area_size)
            y_coordinate = random.uniform(0, area_size)
        substation_positions[i] = (x_coordinate, y_coordinate)
        
    # Establish MST to guarantee a connected network
    mst_edges = build_MST(plant_id, substation_positions)
    
    # Build non-MST edges for redundancy and realism
    kNN_edges = set()
    num_nodes = len(substation_positions)

    for u in range(num_nodes):
        # Build a list of (neighbor, distance) for all other substations
        distance_list = []
        for v in range(num_nodes):
            if v != u:
                d = distance(u, v)
                distance_list.append((v, d))

        # Sort nodes by distance from u
        distance_list.sort(key=lambda pair: pair[1])

        # Select the k closest neighbors
        for index in range(k_nearest):
            v, d = distance_list[index]

            # Normalize ordering for undirected edge
            u2 = min(u, v)
            v2 = max(u, v)

            kNN_edges.add((u2, v2, d))
    
    # Integrate MST and KNN edges
    all_edges = set()

    # Add MST edges
    for (u, v, d) in mst_edges:
        all_edges.add((u, v, d))
    
    # Add KNN edges
    for (u, v, d) in kNN_edges:
        all_edges.add((u, v, d))
        
    # Use random cost multipliers to convert raw Euclidean distances into edge costs
    final_edges = []

    for (u, v, d) in all_edges:
        # Random multiplier: cost-per-mile variation
        multiplier = random.uniform(min_cost_multiplier, max_cost_multiplier)
    
        cost = d * multiplier
    
        # Store both cost and raw distance
        final_edges.append((u, v, d, cost))
    
    # Write graph content to file
    with open(file_name, "w") as f:
        f.write(f"{num_nodes} {plant_id}\n")
    
        # Write each edge as "u v d cost"
        for (u, v, d, cost) in final_edges:
            f.write(f"{u} {v} {d} {cost}\n")
    
    
    # Write positions of each substation in network to separate file for later plotting and analytics
    position_file = file_name.replace(".txt", "_positions.txt")

    with open(position_file, "w") as f:
        for node in range(num_nodes):
            x, y = substation_positions[node]
            f.write(f"{node} {x} {y}\n")
        
    

In [91]:
make_graph(200, seed=42)